# Workfile Examples

Let's run a workflow with workflow variable input and download the results locally.

In [1]:
username = 'demoadmin'
password = 'password'
host = "10.10.0.204"
port = "8080"

test_workflow_name = "WineData"
test_workspace_name = "APITests"

In [2]:
import api as AlpineAPI
from pprint import pprint

In [3]:
session = AlpineAPI.Alpine(host, port, username, password)

## Finding our workflow

In [4]:
test_workspace_id = session.workspace.get_id(test_workspace_name)
all_workfiles = session.workfile.get_list(test_workspace_id)
pprint(all_workfiles, depth = 2)
len(all_workfiles)

[{u'associated_worklets': [],
  u'comment_count': 0,
  u'complete_json': True,
  u'description': u'',
  u'entity_subtype': u'alpine',
  u'entity_type': u'workfile',
  u'file_name': u'Test workflow',
  u'file_type': u'work_flow',
  u'id': 7,
  u'is_deleted': None,
  u'latest_version_id': None,
  u'link_url': None,
  u'owner': {...},
  u'recent_comments': [],
  u'status': u'idle',
  u'tags': [],
  u'user_modified_at': u'2016-12-20T00:41:27Z',
  u'version_info': {...},
  u'workspace': {...}},
 {u'associated_worklets': [],
  u'comment_count': 0,
  u'complete_json': True,
  u'description': u'',
  u'entity_subtype': u'alpine',
  u'entity_type': u'workfile',
  u'file_name': u'WineData',
  u'file_type': u'work_flow',
  u'id': 375,
  u'is_deleted': None,
  u'latest_version_id': None,
  u'link_url': None,
  u'owner': {...},
  u'recent_comments': [],
  u'status': u'idle',
  u'tags': [],
  u'user_modified_at': u'2017-02-07T22:28:29Z',
  u'version_info': {...},
  u'workspace': {...}}]


2

In [5]:
test_workflow_id = session.workfile.get_id(test_workflow_name, test_workspace_id)
single_workfile_info = session.workfile.get(test_workflow_id)
pprint(single_workfile_info, depth = 2)

{u'response': {u'associated_worklets': [],
               u'comment_count': 0,
               u'complete_json': True,
               u'dataset_ids': [],
               u'description': u'',
               u'entity_subtype': u'alpine',
               u'entity_type': u'workfile',
               u'execution_locations': [...],
               u'file_name': u'WineData',
               u'file_type': u'work_flow',
               u'id': 375,
               u'is_deleted': None,
               u'latest_version_id': None,
               u'link_url': None,
               u'owner': {...},
               u'recent_comments': [],
               u'status': u'idle',
               u'tags': [],
               u'user_modified_at': u'2017-02-07T22:28:29Z',
               u'version_info': {...},
               u'workspace': {...}}}


In [6]:
test_workflow_id

375

## Running a workflow

In [11]:
process_id = session.workfile.process.run(test_workflow_id)

In [7]:
session.workfile.process.query_status(process_id)

'WORKING'

In [8]:
session.workfile.process.wait_until_finished(process_id, verbose=True, query_time=5, timeout=100)

Workflow in progress for ~35.33 seconds.      

'FINISHED'

## Downloading the flow results

In [17]:
flow_results_all = session.workfile.process.download_results(test_workflow_id, process_id)

In [18]:
session.workfile.process.get_metadata(flow_results)

{u'endTime': u'2017-02-27T15:14:30.830-0800',
 u'executeUser': u'665',
 u'noOfError': 0,
 u'noOfNodesProcessed': 3,
 u'processId': u'52fa392c-c28a-42f8-b36b-3ddec918b01c',
 u'startTime': u'2017-02-27T15:13:48.198-0800',
 u'status': u'SUCCESS',
 u'workflowId': u'375',
 u'workflowName': u'WineData'}

In [ ]:
operator_data = session.workfile.process.find_operator('Row Filter', flow_results);
pprint(operator_data, depth = 2)

In [ ]:
pprint(session.workfile.process.get_flow_metadata(flow_results))

## Using workflow variables

After properly defining a workflow variable in Alpine UI, a user can inject different values into a workflow. In this workflow I've defined a workflow variable that is being used in a Row Filter operator.

In [15]:
variables = [{"name": "@row_filter", "value": "15"}]
process_id = session.workfile.process.run(test_workflow_id, variables=variables)
session.workfile.process.wait_until_finished(process_id, verbose=True, query_time=5, timeout=100)

Workflow in progress for ~40.32 seconds.       

'FINISHED'

In [17]:
flow_results = session.workfile.process.download_results(test_workflow_id, process_id)
ss_results = session.workfile.process.find_operator("Summary Statistics", flow_results)
pprint(ss_results['visualData']['items'][0]['count'])

u'19'
